Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [12]:
import math
from copy import copy
from functools import reduce
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse

In [13]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [14]:
x = make_set_covering_problem(100, 100, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


In [23]:
PROBLEM_SIZE = 1000
NUM_SETS = 1000
SETS_TRUE_PROBABILITY = 0.3

In [24]:
SETS = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, SETS_TRUE_PROBABILITY)

In [25]:
SETS.A

/Users/dimitrimasetta/Library/Caches/pypoetry/virtualenvs/ci-tWKgAdaj-py3.11/lib/python3.11/site-packages/scipy/sparse/_base.py:719: VisibleDeprecationWarning: `.A` is deprecated and will be removed in v1.13.0. Use `.toarray()` instead.
  warn(np.VisibleDeprecationWarning(


array([[False,  True, False, ..., False, False, False],
       [False,  True,  True, ..., False, False, False],
       [False, False,  True, ..., False, False,  True],
       ...,
       [False,  True, False, ..., False, False, False],
       [False,  True,  True, ...,  True, False,  True],
       [False, False,  True, ..., False, False, False]])

In [26]:
def fitness(state):
    result_coverage = coverage(state)
    return -(state.sum()+ (result_coverage == False).sum() *  PROBLEM_SIZE)   

def coverage(state):
    return (SETS * state).sum(axis=0)

In [27]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [28]:
def tweak_and_feat(state, state_fitness):
    index = randint(0, NUM_SETS - 1)
    state[index] = not state[index]
    new_fitness = fitness(state)
    if new_fitness > state_fitness:
        return True, state, new_fitness
    else:
        state[index] = not state[index]
        return False, state, state_fitness

# Hill climber

In [29]:
# random() < 0.005
current_state = [False for _ in range(NUM_SETS)]
current_state = np.expand_dims(current_state, 0).transpose()
fitness_current = fitness(current_state)
print(f"Start fitness {fitness_current}")

max_step = 5_000
count_steady = max_step/10
count_equal = 0

Start fitness -1000000


In [30]:
for step in range(max_step):
    count_equal += 1
    res, st, fit = tweak_and_feat(current_state, fitness_current)
    if res:
        count_equal = 0
        current_state, fitness_current = st, fit
        print(f"Fit {fitness_current} Step {step}")
    if count_equal > count_steady:
        print("steady state reached")
        break

print(f"Resolved {(coverage(current_state) == False).sum() == 0} with {current_state.sum()} in {step} step")
coverage(current_state)

Fit -679001 Step 0
Fit -489002 Step 1
Fit -342003 Step 2
Fit -245004 Step 3
Fit -192005 Step 4
Fit -126006 Step 5
Fit -91007 Step 6
Fit -64008 Step 7
Fit -45009 Step 8
Fit -26010 Step 9
Fit -14011 Step 10
Fit -13012 Step 11
Fit -10013 Step 12
Fit -7014 Step 13
Fit -5015 Step 14
Fit -4016 Step 16
Fit -2017 Step 17
Fit -1018 Step 19
Fit -19 Step 21
Fit -18 Step 91
Fit -17 Step 95
Fit -16 Step 96
steady state reached
Resolved True with 16 in 597 step


array([ 4,  6,  3,  6,  6,  1,  4,  8,  7,  7,  6,  4,  6,  9,  5,  2,  7,
        4,  4,  3,  5,  6,  5,  5,  5,  2,  4,  6,  4,  7,  7,  2,  5,  5,
        3,  2,  6,  4,  8,  1,  4,  3,  3,  4,  2,  5,  7,  4,  6,  7,  6,
        2,  6,  7,  6,  8,  7,  5,  3,  4,  7,  6,  5,  4,  5,  4,  2,  6,
        7,  7,  6,  6,  4,  7,  4,  7,  4,  3,  6,  4,  9,  7,  4,  5,  4,
        8,  3,  5,  3,  3,  5,  5,  4,  6,  2,  6,  3,  5,  4,  6,  6,  6,
        2,  7,  6,  1,  5,  7,  4,  7,  3,  4,  2,  3,  6,  4,  6,  5,  8,
        5,  4,  4,  7,  1,  5,  3,  6,  5,  4,  2,  5,  4,  3,  4,  5,  4,
        5,  3,  6,  6,  2,  1,  8,  4,  7,  7,  3,  6,  5,  6,  7,  6,  3,
        6,  6,  4,  1,  3,  3,  5,  5,  5,  9, 11,  5,  6,  6,  6,  5,  4,
        1,  7,  6,  8, 10,  4,  5,  3,  6, 10,  3,  8,  4,  9,  2,  6,  8,
        9,  5,  4,  5,  3,  3,  6,  4,  3,  7,  8,  2,  4,  3,  8,  7,  4,
        6,  4,  6,  2,  5,  2,  4,  3,  4,  1,  5,  1,  6,  4,  6,  8,  2,
        7,  3,  7,  9,  5

# Hill climber best of three

In [35]:
current_state = [False for _ in range(NUM_SETS)]
current_state = np.expand_dims(current_state, 0).transpose()
fitness_current = fitness(current_state)
print(f"Start fitness {fitness_current}")

max_step = 5_000
count_steady = max_step/10
count_equal = 0

Start fitness -1000000


In [36]:
for step in range(max_step):
    count_equal += 1
    state_new = tweak(current_state)
    fitness_new = fitness(state_new)
    for _ in range(4):
        s_new = tweak(current_state)
        f_new = fitness(state_new)
        if f_new > fitness_new:
            state_new, fitness_new = s_new, f_new
    
    if fitness_new > fitness_current:
        count_equal = 0
        current_state, fitness_current = state_new, fitness_new
        print(f"Fit {fitness_current} Step {step}")
    if count_equal > count_steady:
        print("steady state reached")
        break

print(f"Resolved {(coverage(current_state) == False).sum() == 0} with {current_state.sum()} in {step * 3} step")
coverage(current_state)

Fit -666001 Step 0
Fit -456002 Step 1
Fit -322003 Step 2
Fit -235004 Step 3
Fit -160005 Step 4
Fit -111006 Step 5
Fit -79007 Step 6
Fit -56008 Step 7
Fit -36009 Step 8
Fit -26010 Step 9
Fit -19011 Step 10
Fit -15012 Step 11
Fit -12013 Step 12
Fit -9014 Step 13
Fit -5015 Step 14
Fit -3016 Step 15
Fit -2017 Step 16
Fit -1018 Step 18
Fit -19 Step 20
Fit -18 Step 22
Fit -17 Step 120
Fit -16 Step 489


KeyboardInterrupt: 

# Simulated annealing

In [42]:
def probability(current_fitness, new_fitness, t):
    if new_fitness > current_fitness:
        return 1
    return math.exp(-(current_fitness - new_fitness) / t) / 2

In [44]:
current_state = [False for _ in range(NUM_SETS)]
current_state = np.expand_dims(current_state, 0).transpose()
fitness_current = fitness(current_state)
print(f"Start fitness {fitness_current}")

max_step = 5_000
count_steady = max_step/10
count_equal = 0

Start fitness -1000000


In [45]:
max_temp = max_step / 10
temperature1 = [1-(i+1)/max_step for i in range(max_step)]
temperature2 = [max_temp/(i+1) for i in range(max_step)]

temperature = temperature2

In [46]:
for step in range(max_step):
    count_equal += 1
    state_new = tweak(current_state)
    fitness_new = fitness(state_new)
    prob = probability(fitness_current, fitness_new, temperature[step])
    if random() < prob:
        count_equal = 0
        current_state, fitness_current = state_new, fitness_new
        print(f"Fit {fitness_current} Step {step} Prob {prob}")
    if count_equal > count_steady:
        print("steady state reached")
        break

print(f"Resolved {(coverage(current_state) == False).sum() == 0} with {current_state.sum()} in {step} step")
coverage(current_state)

Fit -707001 Step 0 Prob 1
Fit -490002 Step 1 Prob 1
Fit -327003 Step 2 Prob 1
Fit -231004 Step 3 Prob 1
Fit -151005 Step 4 Prob 1
Fit -106006 Step 5 Prob 1
Fit -74007 Step 6 Prob 1
Fit -55008 Step 7 Prob 1
Fit -34009 Step 8 Prob 1
Fit -28010 Step 9 Prob 1
Fit -21011 Step 10 Prob 1
Fit -13012 Step 11 Prob 1
Fit -10013 Step 12 Prob 1
Fit -9014 Step 13 Prob 1
Fit -7015 Step 14 Prob 1
Fit -5016 Step 15 Prob 1
Fit -3017 Step 16 Prob 1
Fit -2018 Step 17 Prob 1
Fit -1019 Step 18 Prob 1
Fit -1020 Step 21 Prob 0.47847697873652334
Fit -1021 Step 23 Prob 0.47656689353875237
Fit -1022 Step 24 Prob 0.475614712250357
Fit -1023 Step 25 Prob 0.47466443342144476
Fit -1024 Step 26 Prob 0.47371605325089916
Fit -25 Step 27 Prob 1
Fit -26 Step 28 Prob 0.47182497371839927
Fit -27 Step 30 Prob 0.46994144339554444
Fit -28 Step 33 Prob 0.46713023678860677
Fit -29 Step 42 Prob 0.45879711561007547
Fit -30 Step 43 Prob 0.4578804383616628
Fit -31 Step 48 Prob 0.45332445187696047
Fit -32 Step 49 Prob 0.452418709017

KeyboardInterrupt: 